In [1]:
import pandas as pd
import altair as alt
import numpy as np
import pycountry_convert as pc
import ipywidgets as widgets

In [2]:
url = 'https://covid.ourworldindata.org/data/ecdc/full_data.csv'
df= pd.read_csv(url,parse_dates=['date'],index_col=['date'])

In [3]:
df=df.reset_index()
alt.data_transformers.disable_max_rows()
print("dernière mise à jour des données {}".format(df.iloc[-1]["date"]))

dernière mise à jour des données 2020-09-02 00:00:00


In [4]:


def country_to_continent(country):
    try :
        country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except :
        return np.nan

In [5]:
df["continent"]=df["location"].apply(country_to_continent)

In [11]:
multi = alt.selection_multi(fields=['location'])
binding_select=alt.binding_select(options=df.continent.unique())
selection = alt.selection_single(fields=["continent"],bind=binding_select)
color=alt.condition(multi,alt.Color("new_deaths:Q",legend=None,aggregate="sum"),alt.value("grey"))

matrix_selection=alt.Chart(df).mark_rect().encode(alt.Y("location",title="",sort="-color"),
                                                  color=color).add_selection(multi).transform_filter(selection).add_selection(selection).properties(width=50,title="Nombre de cas (selectionner un ou plusieurs pays (shift + click))")

matrix_text=alt.Chart(df).mark_text().encode(alt.Y("location",title="",sort="-text"),text="sum(new_deaths)").transform_filter(selection)
matrix_selector=matrix_selection + matrix_text 

In [16]:
start_date = df.date.min()
end_date = df.date.max()

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

def test(date):
    multi = alt.selection_multi(fields=['location'])
    binding_select=alt.binding_select(options=df.continent.unique())
    selection = alt.selection_single(fields=["continent"],bind=binding_select)
    color=alt.condition(multi,alt.Color("new_deaths:Q",legend=None,aggregate="sum"),alt.value("grey"))
    matrix_selection=alt.Chart(df.loc[df["date"]>date[0]].loc[df["date"]<date[1]]).mark_rect().encode(alt.Y("location",title="",sort="-color"),
                                                  color=color).add_selection(multi).transform_filter(selection).add_selection(selection).properties(width=50,title="Nombre de cas (selectionner un ou plusieurs pays (shift + click))")
    matrix_text=alt.Chart(df.loc[df["date"]>date[0]].loc[df["date"]<date[1]]).mark_text().encode(alt.Y("location",title="",sort="-text"),text="sum(new_deaths)").transform_filter(selection)
    matrix_selector=matrix_selection + matrix_text 
    
    base_courbe = alt.Chart(df.loc[df["date"]>date[0]].loc[df["date"]<date[1]]).mark_line().encode(alt.X("date:T"),
                      color="location")
    new_case_and_death = base_courbe.encode(alt.Y("new_deaths")) | base_courbe.encode(alt.Y("new_cases"))
    total_new_case_and_death = base_courbe.encode(alt.Y("total_deaths")) | base_courbe.encode(alt.Y("total_cases"))
    courbe = new_case_and_death & total_new_case_and_death
    return  matrix_selector | courbe
widgets.interact(test,date=selection_range_slider)

interactive(children=(SelectionRangeSlider(description='Dates', index=(0, 246), layout=Layout(width='500px'), …

<function __main__.test(date)>